In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import sys
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

import numpy as np
from numpy import zeros,ones,linalg
from numpy.linalg import norm,cholesky,inv
from sklearn.cross_validation import train_test_split
import mosek
import math
import sys
import time 

from skimage import measure
from skimage.draw import ellipsoid

c = [10,10,10]
r=10

# overfit/underfit param
v=.000000001

# Define a stream printer to grab output from MOSEK
def streamprinter(text):
#     sys.stdout.write(text)
#     sys.stdout.flush()
    pass

def incomplete_cholesky_decomp(K,G):
    start = time.time()

    assert K.shape[0] == K.shape[1]
    n = K.shape[0]

    k=-1

    np.fill_diagonal(G[0:n,0:n], np.diagonal(K[0:n,0:n]))
    
    for i in range(n):
        print 'loop',i
        if i < 100 and np.sum(np.diagonal(G[i:n,i:n])) > 1.:
            j = np.argmax(np.diagonal(G[i:n,i:n]))+i

            temp = K.T[0:n,i].copy()
            K.T[0:n,i] = K.T[0:n,j]
            K.T[0:n,j] = temp

            temp = K.T[i,0:n].copy()
            K.T[i,0:n] = K.T[j,0:n]
            K.T[j,0:n] = temp

            temp = G[i,0:i+1].copy()
            G[i,0:i+1] = G[j,0:i+1]
            G[j,0:i+1] = temp

            G[i,i] = math.sqrt(K[i,i])

            G[i+1:n,i] = 1/G[i,i]*(K[i+1:n,i]- np.dot(G[i+1:n,0:i],G[i,0:i]) )

            
            np.fill_diagonal(G[i:n,i:n], np.diagonal(K[i:n,i:n]) - np.sum(np.power(G[i:n,0:i],2),axis=1))
        else:
            k=i
            break

    end = time.time()
#     if end - start > .1:
    print 'incomplete_cholesky_decomp:', end - start, 'sec'
    return G,k


def kernel(x1,x2):
    return math.exp(-1*math.pow(linalg.norm(x1-x2),2)/(2*math.pow(sigma,
                                                                  2)))

def kernel_vect(x_list,x2):
    return np.exp(-1*np.power(norm(x_list-x2,axis=1),2 )/(2*math.pow(sigma, 2)))

approx_avg = []
true_avg = []
k_val = []
for data_ratio in [13]:#15, 14, 13]:
    for is_approx in [False]: #True
        sigma = .01

        x = []
        f = open('bunny.obj.txt')
        for line in f:
            line = line.strip()
            if line != '' and line[0] != '#':
                line_split = line.split()
                if len(line_split)==4 and line_split[0] == 'v':
                    x.append(line_split[1:])
        print 'read file'
        x = np.array(x)
        x=x.astype(np.float)
        x = sorted(x, key=lambda a_entry: a_entry[0]) 
        x = np.array(x)
        x = x[data_ratio*x.shape[0]/16:,:]
        # y = np.ones(len(x))
        m = len(x)
        print 'points:',m
        # x,ignore_x,y,ignore_y=train_test_split(x,y,train_size=0.02,random_state=8)
        m = len(x)

        K = np.zeros((m,m))
        for i in range(m):
            K[i,:] = kernel_vect(x,x[i])
        #     for j in range(m):
        #         K[i,j] = kernel(x[i],x[j])
        print 'got K'

        if is_approx:
            G = np.zeros((m,m))
            G,k = incomplete_cholesky_decomp(K.copy(),G)
            print 'k',k,'m',m
            k_val.append(k)
        # sys.exit()

        x0_max = np.amax(x[:,0])+0.1
        x0_min = np.amin(x[:,0])-0.1
        x1_max = np.amax(x[:,1])+0.1
        x1_min = np.amin(x[:,1])-0.1
        x2_max = np.amax(x[:,2])+0.1
        x2_min = np.amin(x[:,2])-0.1


        print x0_max 
        print x0_min 
        print x1_max 
        print x1_min 
        print x2_max 
        print x2_min 

        # print 'points:',m

        # Open MOSEK and create an environment and task
        # Make a MOSEK environment
        env = mosek.Env ()
        # Attach a printer to the environment
        env.set_Stream (mosek.streamtype.log, streamprinter)
        # Create a task
        task = env.Task()
        task.set_Stream (mosek.streamtype.log, streamprinter)
        # Set up and input bounds and linear coefficients
        bkc   = [ mosek.boundkey.fx]
        blc   = [ 1.0 ]
        buc   = [ 1.0 ]

        bkx=[]
        for i in range(m):
            bkx.append(mosek.boundkey.ra)
        bux=(ones((m))) * (1/(v*m))
        blx=(ones((m))) * (-1/(v*m))
        c=(zeros((m)))

        numvar = len(bkx)
        numcon = len(bkc)

        # Append 'numcon' empty constraints.
        # The constraints will initially have no bounds.
        task.appendcons(numcon)

        # Append 'numvar' variables.
        # The variables will initially be fixed at zero (x=0).
        task.appendvars(numvar)

        for j in range(numvar):
            # Set the linear term c_j in the objective.
            task.putcj(j,c[j])
            # Set the bounds on variable j
            # blx[j] <= x_j <= bux[j]
            task.putbound(mosek.accmode.var,j,bkx[j],blx[j],bux[j])

            # Input i, j of A
            task.putaij(0, j, 1)

        for i in range(numcon):
            task.putbound(mosek.accmode.con,i,bkc[i],blc[i],buc[i])

        # Input the objective sense (minimize/maximize)
        task.putobjsense(mosek.objsense.maximize)

        # Set up and input quadratic objective
        for i in range(m):
            for j in range(i+1):
                if is_approx:
                    task.putqobjij(i,j,np.dot(G[i,j],G[i,j].T))
                else:
                    task.putqobjij(i,j,K[i,j])

        task.putobjsense(mosek.objsense.minimize)

        # Optimize
        task.optimize()
        # Print a summary containing information
        # about the solution for debugging purposes
        task.solutionsummary(mosek.streamtype.msg)

        prosta = task.getprosta(mosek.soltype.itr)
        solsta = task.getsolsta(mosek.soltype.itr)

        # Output a solution
        xx = zeros(numvar, float)
        task.getxx(mosek.soltype.itr, xx)

        # if solsta == mosek.solsta.optimal or solsta == mosek.solsta.near_optimal:
        #     print("Optimal solution: %s" % xx)
        # elif solsta == mosek.solsta.dual_infeas_cer:
        #     print("Primal or dual infeasibility.\n")
        # elif solsta == mosek.solsta.prim_infeas_cer:
        #     print("Primal or dual infeasibility.\n")
        # elif solsta == mosek.solsta.near_dual_infeas_cer:
        #     print("Primal or dual infeasibility.\n")
        # elif  solsta == mosek.solsta.near_prim_infeas_cer:
        #     print("Primal or dual infeasibility.\n")
        # elif mosek.solsta.unknown:
        #     print("Unknown solution status")
        # else:
        #     print("Other solution status")

        alpha = xx

        x_test = x[0] #test at x
        w_x=0
        for i in range(m):
            w_x = w_x + (alpha[i]*kernel(x[i],x_test))

        # print 'w = ',w_x

        num_steps = 20
        x0_range = np.linspace(x0_min, x0_max,num_steps)
        x1_range = np.linspace(x1_min, x1_max,num_steps)
        x2_range = np.linspace(x2_min, x2_max,num_steps)

        # xx,yy,zz = np.meshgrid(x0_range,x1_range,x2_range)

        # print xx,yy,zz

        def rkhs_eval(x_temp):
            w_x=0
            for i in range(m):
                w_x = w_x + (alpha[i]*kernel(x[i],x_temp))
            return w_x

        losses = []
        for i in range(m):
            losses.append(rkhs_eval(x[i]))
        losses = np.asarray(losses)
        print 'losses avg -> ',(np.average( losses ))
        if is_approx:
            approx_avg.append(np.average( losses ))
        else:
            true_avg.append(np.average( losses ))
        print 'losses min -> ',(np.amin( losses ))
        print 'losses min -> ',(np.argmin( losses ))
        print 'losses min -> ',x[(np.argmin( losses ))]
        print 'losses max -> ',(np.amax( losses ))
        print 'losses max -> ',(np.argmax( losses ))
        print 'losses max -> ',x[(np.argmax( losses ))]
        print 'v',v
        print 'sigma',sigma



In [ ]:
rho = (np.amax(losses)+np.amin(losses))/2

data = np.zeros((num_steps,num_steps,num_steps))
for i in range(num_steps):
    for j in range(num_steps):
        for k in range(num_steps):
            x_temp = np.asarray([x0_range[i],
                                x1_range[j],
                                x2_range[k]])
            data[i,j,k] = rkhs_eval(x_temp) - rho

delta=0
print 'delta',delta
print 'rho',rho
print 'np.abs(data - delta) < .1 -> ',(np.where(np.abs(data - delta) < .1)[0].shape)
print 'np.abs(data - delta) < .01 -> ',(np.where(np.abs(data - delta) < .01)[0].shape)
print 'np.abs(data - delta) < .001 -> ',(np.where(np.abs(data - delta) < .001)[0].shape)
print 'np.abs(data - delta) < .0001 -> ',(np.where(np.abs(data - delta) < .0001)[0].shape)
print 'data < delta -> ',(np.where(data < delta )[0].shape)
print 'data > delta -> ',(np.where(data > delta )[0].shape)
print 'data < 0 -> ',(np.where( data < 0)[0].shape)
print 'data == 0 -> ',(np.where( data == 0)[0].shape)
print 'data > 0 -> ',(np.where( data > 0)[0].shape)
print 'min -> ',(np.amin( data ))
print 'max -> ',(np.amax( data ))            
            

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import sys
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

from skimage import measure
from skimage.draw import ellipsoid

# print data

# Use marching cubes to obtain the surface mesh of these ellipsoids
verts, faces = measure.marching_cubes(data, 0)

# # Display resulting triangular mesh using Matplotlib.This can also be done
# # with mayavi (see skimage.measure.marching_cubes docstring).
# fig = plt.figure(figsize=(10, 12))
# ax = fig.add_subplot(111, projection='3d')

# # Fancy indexing: `verts[faces]` to generate a collection of triangles
# mesh = Poly3DCollection(verts[faces])
# ax.add_collection3d(mesh)

# ax.set_xlabel("x-axis: a = 6 per ellipsoid")
# ax.set_ylabel("y-axis: b = 10")
# ax.set_zlabel("z-axis: c = 16")

# ax.set_xlim(0, 10)  # a = 6 (times two for 2nd ellipsoid)
# ax.set_ylim(0, 10)  # b = 10
# ax.set_zlim(0, 10)  # c = 16

# plt.show()


In [ ]:
fig = plt.figure(figsize=(10, 12))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x[:,0],x[:,2],x[:,1])
ax.view_init(elev=20., azim=240)
plt.show()
print len(x)

In [ ]:

print verts

# Display resulting triangular mesh using Matplotlib.This can also be done
# with mayavi (see skimage.measure.marching_cubes docstring).
fig = plt.figure(figsize=(10, 12))
ax = fig.add_subplot(111, projection='3d')

# Fancy indexing: `verts[faces]` to generate a collection of triangles
mesh = Poly3DCollection(verts[faces])
ax.add_collection3d(mesh)

ax.set_xlabel("x-axis: a = 6 per ellipsoid")
ax.set_ylabel("y-axis: b = 10")
ax.set_zlabel("z-axis: c = 16")

ax.set_xlim(np.min(verts)-.1, np.max(verts)+.1)  # a = 6 (times two for 2nd ellipsoid)
ax.set_ylim(np.min(verts)-.1, np.max(verts)+.1)  # b = 10
ax.set_zlim(np.min(verts)-.1, np.max(verts)+.1)  # c = 16

plt.show()
# print x0_max 
# print x0_min 
# print x1_max 
# print x1_min 
# print x2_max 
# print x2_min 

# # print data
# print data.shape

# print alpha
